In [1]:
import cv2 as cv
from PIL import Image, ImageEnhance
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [2]:
def display_bgr2rgp(img):
    rgb_img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.figure(figsize=(10,10))
    plt.imshow(rgb_img)
    plt.show()
    
def preprocess(img, factor=2):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = Image.fromarray(img)
    enhancer = ImageEnhance.Sharpness(img).enhance(factor)
    if gray.std() < 30:
        enhancer = ImageEnhance.Contrast(enhancer).enhance(factor)
    return np.array(enhancer)

def cell_in_same_row(c1, c2):
    c1_center = c1[1] + c1[3] - c1[3] / 2
    c2_bottom = c2[1] + c2[3]
    c2_top = c2[1]
    return c2_top < c1_center < c2_bottom

def avg_height_of_center(row):
    centers = [y + h - h / 2 for x, y, w, h in row]
    return sum(centers) / len(centers)

In [3]:
def cell_detection(img):
    """
    Output is location of cells (list). The index of the list is the same as the index of the cells in the table.
    """
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    cv.rectangle(gray, (0,0), gray.shape[::-1], 0, 2)
    blur = cv.GaussianBlur(gray, (3,3), 0, 0)
    thresh = cv.adaptiveThreshold(~blur,255,cv.ADAPTIVE_THRESH_MEAN_C,cv.THRESH_BINARY,5,-2,)
    
    vertical = horizontal = thresh.copy()
    img_height, img_width = horizontal.shape
    SCALE = 3
    # get vertical and horizontal edges
    horizontal_kernel = cv.getStructuringElement(cv.MORPH_RECT, (int(img_width / SCALE), 1))
    horizontally_opened = cv.morphologyEx(thresh, cv.MORPH_OPEN, horizontal_kernel)
    vertical_kernel = cv.getStructuringElement(cv.MORPH_RECT, (1, int(img_height / SCALE)))
    vertically_opened = cv.morphologyEx(thresh, cv.MORPH_OPEN, vertical_kernel)
    # mask is border table structure (no text)
    mask = horizontally_opened + vertically_opened
    mask = cv.dilate(mask, cv.getStructuringElement(cv.MORPH_RECT, (3,3)))
    
    # finding cell
    contours, hierarchy = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    perimeter_lengths = [cv.arcLength(c, True) for c in contours]
    epsilons = [0.1 * p for p in perimeter_lengths]
    approx_polys = [cv.approxPolyDP(c, e, True) for c, e in zip(contours, epsilons)]

    approx_rects = [p for p in approx_polys if len(p) == 4]
    bounding_rects = [cv.boundingRect(a) for a in approx_polys]
    
    MIN_RECT_WIDTH = 40
    MIN_RECT_HEIGHT = 10
    bounding_rects = [r for r in bounding_rects if MIN_RECT_WIDTH < r[2] and MIN_RECT_HEIGHT < r[3]]
    #largest is the total table
    largest_rect = max(bounding_rects, key=lambda r: r[2] * r[3])
    bounding_rects = [b for b in bounding_rects if b is not largest_rect]
    cells = [c for c in bounding_rects]
    
    # ordering the cells
    rows = []
    while cells:
        first = cells[0]
        rest = cells[1:]
        cells_in_same_row = sorted([c for c in rest if cell_in_same_row(c, first)], key=lambda c: c[0])

        row_cells = sorted([first] + cells_in_same_row, key=lambda c: c[0])
        rows.append(row_cells)
        cells = [c for c in rest if not cell_in_same_row(c, first)]
        
    rows.sort(key=avg_height_of_center)
    
    new_rows = []
    for row in rows:
        new_row_cell = []
        for cell in row:
            cx, cy, cw, ch = cell
            cell_thresh = thresh[cy:cy+ch, cx:cx+cw]
                
            ver = hor = cell_thresh.copy()
            ih, iw = hor.shape
    
            hor_kernel = cv.getStructuringElement(cv.MORPH_RECT, (int(iw/3), 1))
            ver_kernel = cv.getStructuringElement(cv.MORPH_RECT, (1, int(ih/3)))
            hor_opened = cv.morphologyEx(hor, cv.MORPH_OPEN, hor_kernel)
            ver_opened = cv.morphologyEx(ver, cv.MORPH_OPEN, ver_kernel)
            
            both = hor_opened + ver_opened
            clean = cell_thresh - both

            kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
            opened = cv.morphologyEx(clean, cv.MORPH_OPEN, kernel)
            opened = cv.dilate(clean, kernel)
            contours, hierarchy = cv.findContours(opened, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
            bounding_rects = [cv.boundingRect(c) for c in contours]
            FIXED_PX = 4
            MIN_CHAR_AREA = 5 * 9
            char_sized_bounding_rects = [(x, y, w, h) for x, y, w, h in bounding_rects if w * h > MIN_CHAR_AREA]
            if char_sized_bounding_rects:
                minx, miny, maxx, maxy = math.inf, math.inf, 0, 0
                for x, y, w, h in char_sized_bounding_rects:
                    minx = min(minx, x)
                    miny = min(miny, y)
                    maxx = max(maxx, x + w)
                    maxy = max(maxy, y + h)
                x, y, w, h = minx, miny, maxx - minx, maxy - miny
                cropped = (cx+x, cy+y, min(iw, w), min(ih, h+FIXED_PX))
            else:
                cropped = (0,0,0,0)
            new_row_cell.append(cropped)
        new_rows.append(new_row_cell)
    return new_rows

In [4]:
def calculate_iou(predict_bndbox, true_bndbox):
    """
    Calculate the IOU between the precdict and the true bounding box
    Predict (x, y, w, h)
    """
    
    x, y, w, h = predict_bndbox
    
    xmin1, ymin1, xmax1, ymax1 = x, y, x+w, y+h
    xmin2, xmax2, ymin2, ymax2 = true_bndbox
    
    
    xmin = max(xmin1, xmin2)
    xmax = min(xmax1, xmax2)
    ymin = max(ymin1, ymin2)
    ymax = min(ymax1, ymax2)
    
    inter_width = max(xmax - xmin + 1, 0)
    inter_height = max(ymax - ymin + 1, 0)
    
    inter_area = inter_width * inter_height
    
    pred_area = w * h
    true_area = (xmax2 -xmin2 + 1) * (ymax2 - ymin2 + 1)
    
    union_area = pred_area + true_area - inter_area
    
    IOU = round(inter_area / union_area, 3)
    return IOU

In [5]:
img_path = "images/48643.png"

img = cv.imread(img_path)
img = preprocess(img, 2)
result = cell_detection(img)
            
            

In [6]:
result

[[(185, 3, 33, 13), (435, 3, 33, 13), (533, 3, 33, 13)],
 [(8, 20, 135, 13), (0, 0, 0, 0), (0, 0, 0, 0)],
 [(7, 37, 25, 13), (412, 37, 80, 14), (528, 37, 65, 14)],
 [(7, 54, 62, 13), (436, 54, 57, 14), (522, 54, 71, 14)],
 [(7, 71, 175, 9), (419, 71, 74, 12), (520, 71, 73, 12)],
 [(7, 87, 47, 13), (425, 88, 67, 12), (533, 88, 61, 12)],
 [(7, 104, 62, 13), (419, 104, 74, 14), (520, 104, 72, 14)],
 [(7, 121, 240, 13), (442, 121, 49, 14), (541, 121, 52, 14)],
 [(7, 138, 143, 12), (0, 0, 0, 0), (0, 0, 0, 0)],
 [(7, 154, 143, 13), (434, 154, 58, 13), (526, 154, 66, 13)],
 [(7, 171, 113, 13), (418, 171, 75, 13), (520, 171, 73, 14)],
 [(16, 194, 102, 19), (412, 202, 81, 16), (515, 195, 79, 17)],
 [(7, 219, 150, 12), (425, 219, 67, 12), (515, 219, 77, 12)],
 [(7, 235, 150, 13), (427, 236, 65, 12), (522, 235, 72, 13)],
 [(16, 258, 102, 19), (412, 266, 81, 17), (513, 259, 81, 17)],
 [(7, 283, 158, 13), (414, 283, 78, 14), (513, 283, 81, 14)],
 [(7, 300, 158, 12), (412, 300, 81, 12), (514, 300, 7

In [7]:
# handling empty cell
def remove_empty_cell(rows):
    
    for row in rows:
        try:
            while True:
                row.remove((0,0,0,0))
        except ValueError: 
            pass
    # change to 1d list 
    result = []
    for row in rows:
        result += row
    return result 
            
result = remove_empty_cell(result)
result

[(185, 3, 33, 13),
 (435, 3, 33, 13),
 (533, 3, 33, 13),
 (8, 20, 135, 13),
 (7, 37, 25, 13),
 (412, 37, 80, 14),
 (528, 37, 65, 14),
 (7, 54, 62, 13),
 (436, 54, 57, 14),
 (522, 54, 71, 14),
 (7, 71, 175, 9),
 (419, 71, 74, 12),
 (520, 71, 73, 12),
 (7, 87, 47, 13),
 (425, 88, 67, 12),
 (533, 88, 61, 12),
 (7, 104, 62, 13),
 (419, 104, 74, 14),
 (520, 104, 72, 14),
 (7, 121, 240, 13),
 (442, 121, 49, 14),
 (541, 121, 52, 14),
 (7, 138, 143, 12),
 (7, 154, 143, 13),
 (434, 154, 58, 13),
 (526, 154, 66, 13),
 (7, 171, 113, 13),
 (418, 171, 75, 13),
 (520, 171, 73, 14),
 (16, 194, 102, 19),
 (412, 202, 81, 16),
 (515, 195, 79, 17),
 (7, 219, 150, 12),
 (425, 219, 67, 12),
 (515, 219, 77, 12),
 (7, 235, 150, 13),
 (427, 236, 65, 12),
 (522, 235, 72, 13),
 (16, 258, 102, 19),
 (412, 266, 81, 17),
 (513, 259, 81, 17),
 (7, 283, 158, 13),
 (414, 283, 78, 14),
 (513, 283, 81, 14),
 (7, 300, 158, 12),
 (412, 300, 81, 12),
 (514, 300, 78, 12),
 (7, 316, 17, 13),
 (7, 339, 100, 20),
 (412, 340, 

In [8]:
data = pd.read_csv('cell data/48643.csv')
data.head(5)

,xmin,xmax,ymin,ymax
0,187,215,4,14
1,437,465,4,14
2,535,563,4,14
3,9,140,21,31
4,8,29,38,48


In [10]:
IOU = []
for idx, r in enumerate(result):
    iou = calculate_iou(r, data.iloc[idx])
    IOU.append(iou)

IOU

[0.744,
 0.744,
 0.744,
 0.827,
 0.745,
 0.756,
 0.69,
 0.819,
 0.744,
 0.764,
 0.985,
 0.88,
 0.892,
 0.81,
 0.876,
 0.836,
 0.819,
 0.754,
 0.764,
 0.839,
 0.754,
 0.755,
 0.897,
 0.828,
 0.802,
 0.808,
 0.824,
 0.801,
 0.764,
 0.554,
 0.483,
 0.614,
 0.904,
 0.889,
 0.881,
 0.835,
 0.888,
 0.811,
 0.539,
 0.468,
 0.615,
 0.83,
 0.755,
 0.747,
 0.899,
 0.871,
 0.893,
 0.747,
 0.533,
 0.596,
 0.778,
 0.829,
 0.81,
 0.889,
 0.819,
 0.818,
 0.8,
 0.747,
 0.756,
 0.877,
 0.871,
 0.883,
 0.828,
 0.828,
 0.538,
 0.423,
 0.623]

In [11]:
sum(IOU) / len(IOU)

0.7721492537313436

In [12]:
sum(np.array(IOU) > 0.5)

64